# `SubwayBiStation-IntervalTime.ipynb`
> API 정보 - https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15001019#tab_layer_detail_function\
> `소모시간` = 도시철도가 어떤 역에서 다른 역까지 이동하는데 걸리는 순수 시간\
> `정차시간` = 도시철도가 어떤 역에서 도착한 후 정지하여 다음 역으로 출발하기 전 까지의 순수 시간\
> `실질배차간격` = `철도사용자의최악대기시간(환승시간제외)` = 철도사용자가 한 역에서 어떤 열차가 지나가는 것을 보고 다음 열차가 올 때까지의 시간
- 두 역에서의 `소모시간`을 데이터
- 한 역에서의 `정차시간`을 데이터

In [1]:
import numpy as np
import pandas as pd
import json
import util

In [2]:
import yaml

# API KEY 불러오기
with open("../security.yaml") as f:
    SYS_CONFIG = yaml.safe_load(f)

## 1. `소모시간`, `정차시간` 을 알기 위해 API로 데이터 Loading

```py
ENCODING        "EUC-KR"
startSn         출발역명:str
startSc	        출발역코드:int      (ex.신평:101, 안평:414)
endSn	        도착역명:str
endSc	        도착역코드:int      (ex.신평:101, 안평:414)
dist	        이동거리:int        단위:0.1 km (ex 16->1.6 km)
time	        이동시간:int        단위:초(sec)
stoppingTime	정차시간:int        단위:초(sec)
exchange        환승구분:str|Null   "Y" => 환승역
                                   "N" => 경전철
                                   ""(공란) => 일반역```


In [3]:
import requests

url = "http://data.humetro.busan.kr/voc/api/open_api_distance.tnn"
params = {"serviceKey": SYS_CONFIG["API_KEY"], "act": "json", "numOfRows": 328}

# GET API REQ
response = requests.get(url, params=params)

In [4]:
# SET custom JSON DECODER (JSON STRING TO PYTHON OBJEC DECODER)
# https://stackoverflow.com/questions/45068797/how-to-convert-string-int-json-into-real-int-with-json-loads
class Decoder(json.JSONDecoder):
    def decode(self, s):
        result = super().decode(
            s
        )  # result = super(Decoder, self).decode(s) for Python 2.x
        return self._decode(result)

    def _decode(self, o):
        # string이면 API에서  Y/N/"" 인 경우로 황승역 여부를 뜻함.
        if isinstance(o, str):
            if o == "Y":
                return True
            elif o == "N":
                return False
            elif o == "":
                return None

            try:
                return int(o)
            except ValueError:
                return o
        # 나머지는 지알아서 하는거임 ㅇㅇ
        elif isinstance(o, dict):
            return {k: self._decode(v) for k, v in o.items()}
        elif isinstance(o, list):
            return [self._decode(v) for v in o]
        else:
            return o

In [5]:
# 따로 정해준 디코더를 통해 API 응답(json)을 파이썬 캑체(dictionary or list)로 만들어줌
df_interstation_info = json.loads(req := response.content.decode("EUC-KR"), cls=Decoder)

# API 응답만 가져오기
df_interstation_info = df_interstation_info["response"]["body"]["item"]
df_interstation_info = pd.DataFrame(df_interstation_info)
column_order_like = [
    "dist",
    "startSn",
    "endSn",
    "exchange",
    "startSc",
    "endSc",
    "stoppingTime",
    "time",
]
df_interstation_info = df_interstation_info.loc[:, column_order_like]  #

# 지하철 청보만 가져오기
subway_column_only_condition = (df_interstation_info["startSc"] < 500) & (
    df_interstation_info["endSc"] < 500
)
df_interstation_info = df_interstation_info[subway_column_only_condition]

# 뽑은 정보 저장하기.
PATH_INTER_STATION_SPEND_TIME = "../trimmed_data/IntervalTimeAPI.pickle"
util.savePickle(df_interstation_info, PATH_INTER_STATION_SPEND_TIME)
interval_time = util.loadPickle(PATH_INTER_STATION_SPEND_TIME)

# 환승정보만 저장하기.
PATH_INTER_STATION_EXCHANGE_TIME = "../trimmed_data/ExchangeTimeAPI.pickle"
df_exchange_time = interval_time[interval_time["exchange"] == True]
util.savePickle(df_exchange_time, PATH_INTER_STATION_EXCHANGE_TIME)
exchange_time = util.loadPickle(PATH_INTER_STATION_EXCHANGE_TIME)

# 체크
[interval_time.head(3), exchange_time.head(3)]

[   dist startSn endSn  exchange  startSc  endSc  stoppingTime  time
 0    17      동매    신평     False      100    101            30   180
 1    12      동매    장림     False      100     99            20   180
 2    17      신평    동매     False      101    100            30   180,
     dist startSn endSn  exchange  startSc  endSc  stoppingTime  time
 40     1      서면    서면      True      119    219             0   120
 50     1      연산    연산      True      123    305             0   180
 56     3      동래    동래      True      125    402             0   360]

## 2. 역 to 역 시간 구하기
- 시작역, 끝역
- 무정차대상역들
  - IF 지정X THEN 일반노선에서 소모 시간 RETURN
```
attributes = [
    "startSn",
    "startSc",
    "endSn",
    "endSc",
    "dist",
    "time",
    "stoppingTime",
    "exchange",
]
```

In [6]:
# 그래프를 만들기위함.
# time은 간선(vertex) startSc---endSc 의 가중치
# stoppingTime은 노드 endSc 의 가중치
# 환승역 일 때의 time 환승 시 소요 시간임.
def fn_export_subway_graph(_df):
    interval_time = _df
    subway_graph = interval_time.loc[
        :, ["exchange", "startSc", "endSc", "stoppingTime", "time","dist"]
    ]
    subway_graph = subway_graph.groupby(by=["startSc", "endSc", "exchange"]).sum()
    print(
        "len(subway_graph) == len(interval_time) : ",
        len(subway_graph) == len(interval_time),
    )

    subway_graph = subway_graph.T.to_dict()
    subway_graph_index = list(subway_graph.keys())

    # This RETURN ( (환승여부,startSc,endSc), (stoppingTime, time) )
    return subway_graph_index, subway_graph


def fn_export_exchange_station_index(idx):
    # 환승정보역 관계를 갖는 index 추출
    exchange_relation_index = []
    for _index in idx:
        _, _, exchange = _index
        if exchange is True:
            exchange_relation_index.append(_index)

    # This RETURN ( (환승여부,startSc,endSc)
    return exchange_relation_index

In [7]:
df = util.loadPickle(util.PATH_INTER_STATION_SPEND_TIME)
subway_graph_index, subway_graph = fn_export_subway_graph(df)
print(subway_graph)
exchange_relation_index = fn_export_exchange_station_index(idx=subway_graph_index)

print(*exchange_relation_index)
print(subway_graph)

len(subway_graph) == len(interval_time) :  True
{(95, 96, False): {'stoppingTime': 20, 'time': 180, 'dist': 14}, (96, 95, False): {'stoppingTime': 20, 'time': 180, 'dist': 14}, (96, 97, False): {'stoppingTime': 20, 'time': 120, 'dist': 12}, (97, 96, False): {'stoppingTime': 20, 'time': 120, 'dist': 12}, (97, 98, False): {'stoppingTime': 20, 'time': 120, 'dist': 11}, (98, 97, False): {'stoppingTime': 20, 'time': 120, 'dist': 11}, (98, 99, False): {'stoppingTime': 20, 'time': 120, 'dist': 8}, (99, 98, False): {'stoppingTime': 20, 'time': 120, 'dist': 8}, (99, 100, False): {'stoppingTime': 20, 'time': 180, 'dist': 12}, (100, 99, False): {'stoppingTime': 20, 'time': 180, 'dist': 12}, (100, 101, False): {'stoppingTime': 30, 'time': 180, 'dist': 17}, (101, 100, False): {'stoppingTime': 30, 'time': 180, 'dist': 17}, (101, 102, False): {'stoppingTime': 0, 'time': 140, 'dist': 16}, (102, 101, False): {'stoppingTime': 30, 'time': 150, 'dist': 16}, (102, 103, False): {'stoppingTime': 30, 'time': 